In [1]:
from __future__ import print_function

#Libraries

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sqlite3
from matplotlib.pyplot import figure


df = pd.read_csv('F:\Schools\SMU\DS7331_MachineLearning\DataSets/owid-covid-data_modified.csv') # read in the csv file

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34965 entries, 0 to 34964
Data columns (total 27 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Unnamed: 0                      34965 non-null  int64  
 1   continent                       34965 non-null  object 
 2   location                        34965 non-null  object 
 3   date                            34965 non-null  object 
 4   new_cases                       34965 non-null  float64
 5   new_cases_smoothed              34965 non-null  float64
 6   new_deaths                      34965 non-null  float64
 7   new_deaths_smoothed             34965 non-null  float64
 8   reproduction_rate               34965 non-null  float64
 9   new_vaccinations_smoothed       34965 non-null  float64
 10  new_people_vaccinated_smoothed  34965 non-null  float64
 11  stringency_index                34965 non-null  float64
 12  population                      

In [15]:
float_df = df.drop(["continent","location","stringency_range","new_cases_range","date"], axis=1)
float_df.head()



,Unnamed: 0,new_cases,new_cases_smoothed,new_deaths,new_deaths_smoothed,reproduction_rate,new_vaccinations_smoothed,new_people_vaccinated_smoothed,stringency_index,population,...,aged_65_older,aged_70_older,gdp_per_capita,cardiovasc_death_rate,diabetes_prevalence,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,new_deaths_range
0,493,1557.0,1526.286,91.0,72.857,0.98,14115.0,13654.0,46.3,39835428.0,...,2.581,1.337,1803.987,597.029,9.59,37.746,0.5,64.83,0.511,high
1,494,1940.0,1509.143,86.0,75.571,0.97,12944.0,12507.0,46.3,39835428.0,...,2.581,1.337,1803.987,597.029,9.59,37.746,0.5,64.83,0.511,high
2,495,1329.0,1480.143,59.0,73.286,0.95,11773.0,11360.0,46.3,39835428.0,...,2.581,1.337,1803.987,597.029,9.59,37.746,0.5,64.83,0.511,high
3,496,1263.0,1504.000,92.0,78.429,0.94,10602.0,10212.0,46.3,39835428.0,...,2.581,1.337,1803.987,597.029,9.59,37.746,0.5,64.83,0.511,high
4,497,1189.0,1455.143,84.0,79.000,0.93,8989.0,8618.0,46.3,39835428.0,...,2.581,1.337,1803.987,597.029,9.59,37.746,0.5,64.83,0.511,high


In [19]:

from sklearn.model_selection import ShuffleSplit

# we want to predict the X and y data as follows:
if 'new_deaths_range' in float_df:
    y = float_df['new_deaths_range'].values # get the labels we want
    del float_df['new_deaths_range'] # get rid of the class label
    X = float_df.values # use everything else to predict!
       ## X and y are now numpy matrices, by calling 'values' on the pandas data frames we
    #    have converted them into simple matrices to use with scikit learn


In [20]:
# to use the cross validation object in scikit learn, grab an instance
#    of the object and set it up. This object will be able to split our data into 
#    training and testing splits
num_cv_iterations = 3
num_instances = len(y)
cv_object = ShuffleSplit(n_splits=num_cv_iterations,
                         test_size  = 0.2)
                         
print(cv_object)
ShuffleSplit(n_splits=3, random_state=None, test_size=0.2, train_size=None)


ShuffleSplit(n_splits=3, random_state=None, test_size=0.2, train_size=None)


ShuffleSplit(n_splits=3, random_state=None, test_size=0.2, train_size=None)

In [12]:
# run logistic regression and vary some parameters
from sklearn.linear_model import LogisticRegression
from sklearn import metrics as mt


In [21]:
# create a reusable logisitic regression object
#   setup the object with different learning parameters and constants
lr_clf = LogisticRegression(penalty='l2', C=1.0, class_weight=None, solver='liblinear' ) # get object


In [23]:
iter_num=0
# the indices are the rows used for training and testing in each iteration
for train_indices, test_indices in cv_object.split(X,y): 
    X_train = X[train_indices]
    y_train = y[train_indices]
    
    X_test = X[test_indices]
    y_test = y[test_indices]

    lr_clf.fit(X_train,y_train)  # train object
    y_hat = lr_clf.predict(X_test) # get test set precitions
    
# accuracy and confusion matrix for this iterations of training/testing
    acc = mt.accuracy_score(y_test,y_hat)
    conf = mt.confusion_matrix(y_test,y_hat)
    print("====Iteration",iter_num," ====")
    print("accuracy", acc )
    print("confusion matrix\n",conf)
    iter_num+=1

====Iteration 0  ====
accuracy 0.6506506506506506
confusion matrix
 [[1763  500   66]
 [ 111 2732    8]
 [ 229 1529   55]]
====Iteration 1  ====
accuracy 0.6705276705276705
confusion matrix
 [[1845  370  107]
 [  79 2724   26]
 [ 245 1477  120]]
====Iteration 2  ====
accuracy 0.6362076362076362
confusion matrix
 [[1691  612    0]
 [  81 2758    0]
 [ 190 1661    0]]


In [26]:
#interpret the weights

# iterate over the coefficients
weights = lr_clf.coef_.T # take transpose to make a column vector
variable_names = float_df.columns
for coef, name in zip(weights,variable_names):
    print(name, 'has weight of', coef[0])


Unnamed: 0 has weight of -1.0965530295879385e-05
new_cases has weight of 0.000935786662848598
new_cases_smoothed has weight of 0.0003065326033667785
new_deaths has weight of 6.55569219659854e-05
new_deaths_smoothed has weight of 4.561222203807255e-05
reproduction_rate has weight of -1.0074399961731265e-06
new_vaccinations_smoothed has weight of -2.9820910807198924e-07
new_people_vaccinated_smoothed has weight of 1.3363030745739395e-06
stringency_index has weight of -2.1826327036171816e-05
population has weight of -2.377208645189386e-09
population_density has weight of -0.0001574832019070133
median_age has weight of -1.152829198423026e-05
aged_65_older has weight of 3.7692328206995657e-07
aged_70_older has weight of 2.3027216122232798e-07
gdp_per_capita has weight of -5.196380902634084e-05
cardiovasc_death_rate has weight of -0.00022888761243282797
diabetes_prevalence has weight of -4.588025724910503e-06
handwashing_facilities has weight of -8.438934164528358e-06
hospital_beds_per_thous

In [27]:
from sklearn.preprocessing import StandardScaler

In [28]:
# scale attributes by the training set
scl_obj = StandardScaler()
scl_obj.fit(X_train) # find scalings for each column that make this zero mean and unit std
# the line of code above only looks at training data to get mean and std and we can use it 
# to transform new feature data

X_train_scaled = scl_obj.transform(X_train) # apply to training


In [31]:
X_test_scaled = scl_obj.transform(X_test) # apply those means and std to the test set (without snooping at the test set values)

# train the model just as before
lr_clf = LogisticRegression(penalty='l2', C=0.05, solver='liblinear') # get object, the 'C' value is less (can you guess why??)
lr_clf.fit(X_train_scaled,y_train)  # train object

y_hat = lr_clf.predict(X_test_scaled) # get test set precitions

acc = mt.accuracy_score(y_test,y_hat)
conf = mt.confusion_matrix(y_test,y_hat)
print('accuracy:', acc )
print(conf )




accuracy: 0.7205777205777206
[[1953  249  101]
 [  57 2499  283]
 [  56 1208  587]]


In [37]:
# sort these attributes and spit them out
zip_vars = zip(lr_clf.coef_.T,float_df.columns) # combine attributes
for coef, name in zip_vars:
    print(name, 'has weight of', coef[0]) # now print them out


Unnamed: 0 has weight of 0.023752774845105323
new_cases has weight of 3.480690217070062
new_cases_smoothed has weight of 0.572312458909966
new_deaths has weight of 10.048282521383507
new_deaths_smoothed has weight of 3.253165590801007
reproduction_rate has weight of -0.21356785378933787
new_vaccinations_smoothed has weight of -0.24817561740102626
new_people_vaccinated_smoothed has weight of 0.09862965570328225
stringency_index has weight of 0.24337571183194903
population has weight of 0.07500954080172478
population_density has weight of -0.16820681636779813
median_age has weight of 0.3359473854605797
aged_65_older has weight of 0.3714276463572733
aged_70_older has weight of -0.3380248578802775
gdp_per_capita has weight of -0.66937233784212
cardiovasc_death_rate has weight of 0.06693684540233664
diabetes_prevalence has weight of -0.12182388867475295
handwashing_facilities has weight of 0.08539514040045483
hospital_beds_per_thousand has weight of 0.027484801421827916
life_expectancy has 